# Create interannual tiled snowDynamics products

## Setup

In [1]:
import pandas as pd
import os
import xarray as xr
import numpy as np
import datetime as dt

import geopandas as gpd # To get HBL tile IDs

import sys
sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/DataAccess/Utilities')
import PreProcess_Utils as pputil

sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/Modelling/Utilities')
import Snow_Utils as snow_util

In [2]:
from dask.distributed import LocalCluster

cluster = LocalCluster(n_workers = 10, threads_per_worker = 10, memory_limit = 0) # Refined through testing
client = cluster.get_client()
client # Can open in other window by clicking 'Dashboard' link below

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 100,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57411,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 100
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:57507,Total threads: 10
Dashboard: http://127.0.0.1:57511/status,Memory: 0 B
Nanny: tcp://127.0.0.1:57414,


2025-02-10 11:08:35,974 - distributed.nanny - ERROR - 
Traceback (most recent call last):
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\utils.py", line 805, in wrapper
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\nanny.py", line 554, in _on_worker_exit
    await self._unregister()
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\nanny.py", line 305, in _unregister
    await wait_for(
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\utils.py", line 1910, in wait_for
    return await fut
           ^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\core.py", line 1259, in send_recv_from_rpc
    return await send_recv(comm=comm, op=key, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mb

In [3]:
tile_stats = pd.read_csv(r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\High Resolution Snow Dynamics\IMS (1 km)\Outputs\tiles_IMS_SnowDynamics4HLS.csv", index_col = 0)
tile_stats

,smin_avg,smax_avg,sstart_avg,send_avg,send_last,sstart_first
0,209.85715,34.500000,4.166666,61.500000,90.0,335.0
1,217.14285,37.166668,3.333333,68.500000,90.0,335.0
2,209.71428,34.666668,10.000000,61.833332,90.0,335.0
3,194.42857,29.833334,35.333332,69.333336,90.0,335.0
4,209.57143,36.833332,5.833334,60.000000,90.0,335.0
...,...,...,...,...,...,...
3386,230.71428,37.833332,-133.285720,230.714280,NaN,NaN
3387,227.14285,39.166668,-136.857150,227.142850,NaN,NaN
3388,224.71428,37.000000,-139.285720,224.714280,NaN,NaN
3389,229.71428,33.666668,-134.285720,229.714280,NaN,NaN


In [4]:
tile_list = list(range(2837,2842)) + list(range(2894,2899)) + list(range(2944,2949)) + list(range(2990,2995)) + list(range(3037,3042)) # North Baffin
#tile_list = [828, 829, 830, 904, 905, 906, 988, 989, 990] # For 3x3 test area only
#tile_list = [823]
#tile_list = gpd.read_file(r'C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\Canada\Tiling\TileSytems\60km\canadaFishnet_CanVec_60km.shp')
#tile_list = tile_list[tile_list['HBL'] == 1].index.tolist()
#tile_list = tile_stats.index.tolist() # for all tiles - Note: only have data for HBL tiles

wy = '1824'

#winterYear_tile = 'C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/HBL/TemporalSnow/Fmask_V1/Products2/winterYear_tile/' 
winterYear_tile = 'F:/HLS/Canada/Tiles60km/snowDynamics_fmask_v1/winterYear/' # winterYear tiled products
interannual_tile = 'F:/HLS/Canada/Tiles60km/snowDynamics_fmask_v1/interannual/2018-2024' # Where to so save interannual tiled products
#'C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/HBL/TemporalSnow/Fmask_V1/Products2/interannual_tile' 

## Workflow

In [5]:
%%time
for tile in tile_list:

    filename = 'HLS_Fmask_snowDynamics' + wy + '_' + str(tile) + '.nc.'

    if not any(filename in nc for nc in os.listdir(interannual_tile)): # If NetCDF file has not yet been created    

        print('---------------------------')
        print('...Tile ' + str(tile_list.index(tile) + 1) + ' (ID: ' + str(tile) + ')' + ' of ' + str(len(tile_list)) + '...')

        # Load winter year snow dynamics cubes
        snowDynamics = xr.open_mfdataset(winterYear_tile + str(tile) + '/*.nc', engine = 'netcdf4').compute()

        print('Loaded winterYear snowDynamics dataset (' + str(len(snowDynamics)) + ' variables, ' + str(snowDynamics.sizes['winterYear']) + ' winterYears).')

        # Identify implausible snow period
        if np.isnan(tile_stats.loc[tile, 'send_last']) == True:
            implausible_snow = []
            print('No implausible snow period for this tile.')
        else: # Week buffer from latest recorded end date and earliest recorded start date
            implausible_snow = [int(tile_stats.loc[tile, 'send_last']) + 7, int(tile_stats.loc[tile, 'sstart_first']) - 7]
            
            date1 = dt.datetime(2023, 12, 31) + dt.timedelta(implausible_snow[0])
            date1 = str(date1.month) + '/' + str(date1.day)
    
            date2 = dt.datetime(2023, 12, 31) + dt.timedelta(implausible_snow[1])
            date2 = str(date2.month) + '/' + str(date2.day)
            
            print('Implausible snow period identified as ' + date1 + ' (doy ' + str(implausible_snow[0]) + ') to ' + date2 + ' (doy ' + str(implausible_snow[1]) + ').')

        # Create interannual snowDynamics dataset using weighted mean
        snowDynamics = snow_util.interannualSnowDynamics(snowDynamics, 
                                                         min_count = 'half',
                                                         products = ['start', 'end', 'length', 'periods', 'status'],
                                                         uncertainty = ['start_u', 'end_u', 'length_u'], # Maybe do not need uncertainty means in future (use quality)
                                                         clearObsPerDay = [], # Not included
                                                         form = 'mean_weighted',
                                                         implausible_snow = implausible_snow,
                                                         sd = False, 
                                                         quality = True,
                                                         best_value = True,
                                                         verbose = False)

        print('Output interannual snowDynamics products ' + str(tuple(list(snowDynamics.keys()))) + '.')

        # Save dataset to NetCDF
        
        pputil.downloadNC(snowDynamics, interannual_tile, filename, form = '')
        print('Interannual dataset saved: ' + filename)

        # Memory management at end
        del snowDynamics

---------------------------
...Tile 1 (ID: 2837) of 25...
Loaded winterYear snowDynamics dataset (8 variables, 6 winterYears).
Implausible snow period identified as 8/6 (doy 219) to 9/2 (doy 246).
Output interannual snowDynamics products ('snow_start_mn', 'snow_end_mn', 'snow_length_mn', 'snow_periods_mn', 'pPerennialSnow', 'pSnowFree', 'snow_start_u_mn', 'snow_start_q_mn', 'snow_start_bv', 'snow_start_bvy', 'snow_start_bvq', 'snow_end_u_mn', 'snow_end_q_mn', 'snow_end_bv', 'snow_end_bvy', 'snow_end_bvq', 'snow_length_u_mn', 'snow_length_q_mn', 'snow_length_bv', 'snow_length_bvy', 'snow_length_bvq').
Interannual dataset saved: HLS_Fmask_snowDynamics1824_2837.nc.
---------------------------
...Tile 2 (ID: 2838) of 25...
Loaded winterYear snowDynamics dataset (8 variables, 6 winterYears).
Implausible snow period identified as 8/8 (doy 221) to 9/4 (doy 248).
Output interannual snowDynamics products ('snow_start_mn', 'snow_end_mn', 'snow_length_mn', 'snow_periods_mn', 'pPerennialSnow', 'pS

In [ ]:
# Pre-compute: 33 seconds
# Dask throughout: Much longer

In [ ]:
client.shutdown()

## Testing

In [ ]:
# tile = 904
# snowDynamics = xr.open_mfdataset(winterYear_tile + str(tile) + '/*.nc', engine = 'netcdf4').compute()
# snowDynamics

In [ ]:
# snowDynamics.rio.crs

In [ ]:
# if np.isnan(tile_stats.loc[tile, 'send_last']) == True:
#     implausible_snow = []
#     print('No implausible snow period for this tile.')
# else: # Week buffer from latest recorded end date and earliest recorded start date
#     implausible_snow = [int(tile_stats.loc[tile, 'send_last']) + 7, int(tile_stats.loc[tile, 'sstart_first']) - 7]
# implausible_snow

In [ ]:
# snowDynamics = snow_util.interannualSnowDynamics(snowDynamics, 
#                                                  min_count = 'half',
#                                                  products = ['start', 'end', 'endSum', 'length', 'periods', 'status'],
#                                                  uncertainty = ['start_u', 'end_u', 'length_u'],
#                                                  clearObsPerDay = ['year', 'season', 'fall', 'peak', 'melt'], 
#                                                  type = 'mean_weighted',
#                                                  implausible_snow = implausible_snow,
#                                                  sd = True, 
#                                                  quality = True,
#                                                  best_value = True,
#                                                  verbose = False)
# snowDynamics

In [ ]:
# pputil.downloadNC(snowDynamics, interannual_tile, 'HLS_Fmask_snowDynamics1823_' + str(tile) + '2.nc', type = '')

In [ ]:
# test = pputil.uploadNC(r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\HBL\TemporalSnow\Fmask_V1\Products2\interannual_tile\HLS_Fmask_snowDynamics1823_9042.nc", type = 'dataset')
# test

In [ ]:
# test = pputil.uploadNC(r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\HBL\TemporalSnow\Fmask_V1\Products2\winterYear_tile\828\HLS_Fmask_snowDynamics_winterYear1819_828.nc", type = 'dataset')
# test

In [ ]:
# test2 = test.reset_coords(names = ['band'], drop = True)
# test2

In [ ]:
# test2.rio.crs

In [ ]:
# tile = 828
# snowDynamics = xr.open_mfdataset(winterYear_tile + str(tile) + '/*.nc', engine = 'netcdf4').compute()
# #snowDynamics
# implausible_snow = [int(tile_stats.loc[tile, 'send_last']) + 7, int(tile_stats.loc[tile, 'sstart_first']) - 7]
# implausible_snow

In [ ]:
# snowDynamics

In [ ]:
# # Create empty xarray Dataset to fill with selected products
# import math
# min_count = math.ceil(snowDynamics.sizes['winterYear'] / 2) # Half, rounded up to nearest int

# crs = snowDynamics.rio.crs # Some xarray functions remove crs 
# snowDynamics_i = xr.Dataset(coords = dict(x = ('x', snowDynamics['x'].values), y = ('y', snowDynamics['y'].values)))
# snowDynamics_i.rio.write_crs(crs, inplace = True) # Reapply crs
# snowDynamics_i

In [ ]:
# valid_count = snowDynamics['snow_start'].notnull().sum(dim = 'winterYear') # Number of valid values per pixel
# snowDynamics['snow_start'] = xr.where(valid_count >= min_count, snowDynamics['snow_start'], np.nan) # Filter by valid_count
# weights_u = 2.718 ** (-0.046 * snowDynamics['snow_start_u']) # -0.046 means 50% weight on 15, exponential
# # Get doys
# early_start = implausible_snow[1] - 365 # Before this date = early start
# #late_start = int(snowDynamics['snow_start'].chunk({'winterYear': -1, 'x': -1, 'y': -1}).quantile(0.95))  # After this = late start 
# late_start = int(snowDynamics['snow_start'].quantile(0.95))  # After this = late start 

# # Get diffs
# early_diff = abs(snowDynamics['snow_start'] - early_start) # Negative (before abs) = Before early start
# late_diff = snowDynamics['snow_start'] - late_start # Positive = After late start

# # Isolate outside
# outside_diff = xr.where(snowDynamics['snow_start'] >= early_start, 0, early_diff) # Add early end days
# outside_diff = xr.where(snowDynamics['snow_start'] <= late_start, outside_diff, late_diff) # Add late end days

# # Get weights
# weights_i1 = 2.718 ** (-0.046 * outside_diff) # -0.046 means 50% weight on 15, exponential

# med_diff = abs(snowDynamics['snow_start'] - snowDynamics['snow_start'].median(dim = 'winterYear', skipna = True))
# weights_i2 = 2.718 ** (-0.046 * med_diff) # -0.046 mean 50% weight on 15, exponential    

# weights_start_c = (weights_u * 0.5) + (weights_i1 * 0.25) + (weights_i2 * 0.25) # 50% weight for uncertainty, 50% for implausibility

# snow_start_mn = snowDynamics['snow_start'].weighted(weights_start_c.fillna(0)).mean(dim = 'winterYear', skipna = True)

# snow_start_mn.rio.write_crs(crs, inplace = True) # Reapply crs
# snowDynamics_i['snow_start_mn'] = snow_start_mn # Add to snowDynamics

# snowDynamics_i

In [ ]:
# snow_start_sd = snowDynamics['snow_start'].std(dim = 'winterYear', skipna = True)
# snow_start_sd.rio.write_crs(crs, inplace = True) # Reapply crs
# snowDynamics_i['snow_start_sd'] = snow_start_sd # Add to snowDynamics
# snowDynamics_i

In [ ]:
# snowDynamics['snow_start_u'] = xr.where(snowDynamics['snow_start'].notnull(), snowDynamics['snow_start_u'], np.nan)
# snow_start_u_mn = snowDynamics['snow_start_u'].weighted(weights_start_c.fillna(0)).mean(dim = 'winterYear', skipna = True)
# snow_start_u_mn.rio.write_crs(crs, inplace = True) # Reapply crs
# snowDynamics_i['snow_start_u_mn'] = snow_start_u_mn # Add to snowDynamics  
# snowDynamics_i

In [ ]:
# snow_start_q_mn = weights_start_c.weighted(weights_start_c.fillna(0)).mean(dim = 'winterYear', skipna = True)
# snow_start_q_mn.rio.write_crs(crs, inplace = True) # Reapply crs
# snowDynamics_i['snow_start_q_mn'] = snow_start_q_mn  
# snowDynamics_i

In [ ]:
# %%time
# snow_start_bvi = weights_start_c.fillna(0).argmax(dim = 'winterYear', skipna = True) # Fill na with 0 weight or get error
# snow_start_bv = snowDynamics['snow_start'].isel(winterYear = snow_start_bvi, drop = True).reset_coords(names = 'winterYear', drop = True) # Don't need new coord
# snow_start_bv
# # snow_start_bv.rio.write_crs(crs, inplace = True) # Reapply crs
# # snowDynamics_i['snow_start_bv'] = snow_start_bv
# # snowDynamics_i